In [1]:
import numpy as np
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt

In [2]:
def evalf(x):
  assert type(x) is np.ndarray and len(x) == 2
  return 400*x[0]**2 + 0.004*x[1]**4

In [3]:
def evalg(x):
  assert type(x) is np.ndarray and len(x) == 2
  return np.array([800*x[0],0.016*x[1]**3])

In [4]:
def evalh(x):
  assert type(x) is np.ndarray and len(x) == 2
  return np.array([[800,0],[0,0.048*x[1]**2]])

In [5]:
def find_minimizer(start_x, tol):
  assert type(start_x) is np.ndarray and len(start_x) == 2
  assert type(tol) is float and tol>=0

  x = start_x
  gradf = evalg(x)
  hesf = evalh(x)
  step_length = 1
  k = 0

  while np.linalg.norm(gradf) > tol:
    x = np.subtract(x,np.multiply(step_length,np.matmul(np.linalg.inv(hesf),gradf)))
    k += 1
    gradf = evalg(x)
    hesf = evalh(x)
    print('x:',x)

  return x, k, evalf(x)

In [6]:
start_x = np.array([2,2])
tol = 1e-9

In [7]:
find_minimizer(start_x, tol)

x: [0.         1.33333333]
x: [0.         0.88888889]
x: [0.         0.59259259]
x: [0.         0.39506173]
x: [0.         0.26337449]
x: [0.         0.17558299]
x: [0.         0.11705533]
x: [0.         0.07803688]
x: [0.         0.05202459]
x: [0.         0.03468306]
x: [0.         0.02312204]
x: [0.         0.01541469]
x: [0.         0.01027646]
x: [0.         0.00685097]
x: [0.         0.00456732]
x: [0.         0.00304488]


(array([0.        , 0.00304488]), 16, 3.4382653805813626e-13)

In [8]:
def compute_steplength_backtracking_scaled_direction(x, gradf, alpha_start, rho, gamma, d_k):
  assert type(x) is np.ndarray and len(x) == 2
  assert type(gradf) is np.ndarray and len(gradf) == 2
  
  alpha = alpha_start

  while evalf(x+alpha*np.matmul(d_k,-gradf)) > evalf(x) + gamma*alpha*(np.matmul(gradf.transpose(),np.matmul(d_k,-gradf))):
    alpha = rho*alpha

  return alpha

In [9]:
def find_minimizer_newton_backtracking(start_x, tol, *args):
  assert type(start_x) is np.ndarray and len(start_x) == 2
  assert type(tol) is float and tol>=0

  x = start_x
  gradf = evalg(x)
  hesf = evalh(x)

  alpha_start = args[0]
  rho = args[1]
  gamma = args[2]

  dk = np.linalg.inv(hesf)

  k = 0

  while np.linalg.norm(gradf) > tol:
    
    step_length = compute_steplength_backtracking_scaled_direction(x,gradf,alpha_start,rho,gamma,dk)
    x = np.subtract(x,np.multiply(step_length,np.matmul(dk,gradf)))

    gradf = evalg(x)
    hesf = evalh(x)
    k += 1
    dk = np.linalg.inv(hesf)
    print('x:',x,'alpha:',step_length)

  return x,k,evalf(x)

In [10]:
find_minimizer_newton_backtracking(start_x, tol, 1.0, 0.5, 0.5)

x: [0.         1.33333333] alpha: 1.0
x: [0.         0.88888889] alpha: 1.0
x: [0.         0.59259259] alpha: 1.0
x: [0.         0.39506173] alpha: 1.0
x: [0.         0.26337449] alpha: 1.0
x: [0.         0.17558299] alpha: 1.0
x: [0.         0.11705533] alpha: 1.0
x: [0.         0.07803688] alpha: 1.0
x: [0.         0.05202459] alpha: 1.0
x: [0.         0.03468306] alpha: 1.0
x: [0.         0.02312204] alpha: 1.0
x: [0.         0.01541469] alpha: 1.0
x: [0.         0.01027646] alpha: 1.0
x: [0.         0.00685097] alpha: 1.0
x: [0.         0.00456732] alpha: 1.0
x: [0.         0.00304488] alpha: 1.0


(array([0.        , 0.00304488]), 16, 3.4382653805813626e-13)

#Ans 3:

For Newton's Method with step length = 1, we get the optimizer in 16 iterations.

For Newton's Method with Backtracking Line Search, we get the optimizer in 16 iterations.

Both the methods take same no. of iterations.

Minimizer for Newton's Method : [0.        , 0.00304488]

Minimum Function Value for Newton's Method : 3.4382653805813626e-13

Minimizer for Newton's Method with backtracking line search : [0.        , 0.00304488]

Minimum function value for Newton's method with backtracking line search : 3.4382653805813626e-13

Both the methods give same minimizer and minimum function value.


In [11]:
def compute_steplength_backtracking_prev(x, gradf, alpha_start, rho, gamma):
  assert type(x) is np.ndarray and len(x) == 2
  assert type(gradf) is np.ndarray and len(gradf) == 2
  
  alpha = alpha_start

  while evalf(x+alpha*-gradf) > evalf(x) + gamma*alpha*np.matmul(gradf.transpose(),-gradf):
    alpha = rho*alpha

  return alpha

In [12]:
def compute_steplength_backtracking_scaled_direction_prev(x, gradf, alpha_start, rho, gamma, d_k):
  assert type(x) is np.ndarray and len(x) == 2
  assert type(gradf) is np.ndarray and len(gradf) == 2
  
  alpha = alpha_start

  while evalf(x+alpha*np.matmul(d_k,(-gradf))) > evalf(x) - gamma*alpha*np.matmul(gradf.transpose(),np.matmul(d_k,-gradf)):
    alpha = rho*alpha

  return alpha

In [13]:
BACKTRACKING_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH_WITH_SCALING = 2

In [14]:
def find_minimizer_prev(start_x, tol, line_search_type, *args):
  assert type(start_x) is np.ndarray and len(start_x) == 2
  assert type(tol) is float and tol>=0 

  x = start_x
  g_x = evalg(x)

  alpha_start = args[0]
  rho = args[1]
  gamma = args[2]
  hes_x = evalh(x)

  if(line_search_type == BACKTRACKING_LINE_SEARCH_WITH_SCALING):
    d_k = np.linalg.inv(hes_x)

  k = 0

  while (np.linalg.norm(g_x) > tol):
  
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking_prev(x,g_x, alpha_start,rho, gamma)
      x = np.subtract(x, np.multiply(step_length,g_x))
      k += 1 
      print('x:',x,'alpha:',step_length)
      g_x = evalg(x) 
      hes_x = evalh(x)
    elif line_search_type == BACKTRACKING_LINE_SEARCH_WITH_SCALING:
      step_length = compute_steplength_backtracking_scaled_direction_prev(x,g_x, alpha_start,rho, gamma, d_k)
      x = np.subtract(x, np.multiply(step_length,np.matmul(d_k,g_x))) 
      k += 1
      print('x:',x,'alpha:',step_length)
      g_x = evalg(x) 
      hes_x = evalh(x)
      d_k = np.linalg.inv(hes_x) 
      
    else:  
      raise ValueError('Line search type unknown. Please check!')
    
    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k,evalf(x)

In [ ]:
find_minimizer_prev(start_x, tol, BACKTRACKING_LINE_SEARCH,1.,.5,.5)

In [16]:
find_minimizer_prev(start_x,tol,BACKTRACKING_LINE_SEARCH_WITH_SCALING,1.,.5,.5)

x: [0.         1.33333333] alpha: 1.0
x: [0.         0.88888889] alpha: 1.0
x: [0.         0.59259259] alpha: 1.0
x: [0.         0.39506173] alpha: 1.0
x: [0.         0.26337449] alpha: 1.0
x: [0.         0.17558299] alpha: 1.0
x: [0.         0.11705533] alpha: 1.0
x: [0.         0.07803688] alpha: 1.0
x: [0.         0.05202459] alpha: 1.0
x: [0.         0.03468306] alpha: 1.0
x: [0.         0.02312204] alpha: 1.0
x: [0.         0.01541469] alpha: 1.0
x: [0.         0.01027646] alpha: 1.0
x: [0.         0.00685097] alpha: 1.0
x: [0.         0.00456732] alpha: 1.0
x: [0.         0.00304488] alpha: 1.0


(array([0.        , 0.00304488]), 16, 3.4382653805813626e-13)

#Ans 4:

For Newton's Method with step length = 1, we get the optimizer in 16 iterations.

For Newton's Method with Backtracking Line Search, we get the optimizer in 16 iterations.

For Gradient Descent without backtracking line search, it does not converge quickly, so I stopped it midway.

For Gradient Descent with backtracking line search, we get the optimizer in 16 iterations.

All the methods except gradient descent without backtracking line search take same no. of iterations.

Minimizer for Newton's Method : [0.        , 0.00304488]

Minimum Function Value for Newton's Method : 3.4382653805813626e-13

Minimizer for Newton's Method with backtracking line search : [0.        , 0.00304488]

Minimum function value for Newton's method with backtracking line search : 3.4382653805813626e-13

Minimizer for Gradient Descent without backtracking line search : [0.        , 0.00304488]

Minimum function Value for Gradient descent without backtracking line search : 3.4382653805813626e-13

All the methods except Gradient Descent without backtracking line search give same minimizer and minimum function value.
